In [ ]:
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import BatchNormalization
import keras


import numpy as np
import glob
import cv2
import os
import matplotlib.pyplot as plt
import h5py

In [ ]:
hdf5_path = '/content/drive/My Drive/monogramdataset.hdf5'
hdf5_pathtest = '/content/drive/My Drive/monogramdatasetvalidation.hdf5'

hdf5_file = h5py.File(hdf5_path, "r")
hdf5_filetest = h5py.File(hdf5_pathtest, "r")

# reshape to be [samples][pixels][width][height]
X_train = hdf5_file.get('train_img').value
y_train = hdf5_file.get('train_labels').value
X_val  = hdf5_filetest.get('val_img').value
y_val  = hdf5_filetest.get('val_labels').value
X_test  = hdf5_filetest.get('test_img').value
y_test  = hdf5_filetest.get('test_labels').value
# X_train = X_train.reshape(X_train.shape[2],X_train.shape[1],X_train.shape[0])
# X_test = X_test.reshape(X_test.shape[2],X_test.shape[1],X_test.shape[0])
hdf5_file.close()
X_train = X_train.reshape(X_train.shape[0], 299, 299,3).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 299, 299,3).astype('float32')
X_val  = X_val.reshape(X_val.shape[0], 299, 299,3).astype('float32')

In [ ]:
# X_train = X_train.reshape(X_train.shape[0], 299, 299,1).astype('float32')
# X_test = X_test.reshape(X_test.shape[0], 299, 299,1).astype('float32')
# X_val  = X_val.reshape(X_val.shape[0], 299, 299,1).astype('float32')

# create base pre-trained model
from keras.applications.xception import Xception
# from keras.applications.inception_resnet_v2 import InceptionResNetV2
base_model = Xception(weights='imagenet', include_top=False)

#add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(200,activation='elu')(x)
x = Dropout(0.4)(x)
x = Dense(170,activation='elu')(x)
predictions = Dense(157,activation='hard_sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# # we use SGD with a low learning rate
from keras.optimizers import Nadam
from keras.optimizers import adam
model.compile(optimizer=Nadam(lr=0.0001), loss='binary_crossentropy',metrics=['accuracy'])
# model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

# # we train our model again (this time fine-tuning the top 2 inception blocks
# # alongside the top Dense layers
# model.fit_generator(...)
model.fit(X_train, y_train, validation_data=(X_val, y_val),shuffle="batch" ,epochs=10, batch_size=10,verbose=1)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0,batch_size=10)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

# # serialize model to JSON
try:
    model_json = model.to_json()
    with open("/content/drive/My Drive/monogrammodelxceptioneludrop299.json", "w") as json_file:
        json_file.write(model_json)
except:
    pass
# serialize weights to HDF5
model.save_weights("/content/drive/My Drive/monogrammodelxceptioneludrop299.h5")
print("Saved model to disk")

In [ ]:
#predict the image and save it

In [ ]:
import numpy as np
from keras.optimizers import adam
from keras.models import model_from_json
import h5py
import glob
import os

In [ ]:
hdf5_path = u'/content/drive/My Drive/monogramcleaning.hdf5'
hdf5_file = h5py.File(hdf5_path, "r")
X_pattern = hdf5_file.get('monogram_img').value
hdf5_file.close()

In [ ]:
json_file = open('/content/drive/My Drive/patternmodelnasnetdrop299.json')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("/content/drive/My Drive/patternmodelnasnetdrop299.h5")

In [ ]:
loaded_model.compile(optimizer=adam(lr=0.00008), loss='binary_crossentropy',metrics=['accuracy'])


In [ ]:
  prediction = loaded_model.predict(X_pattern,batch_size=1,verbose=1)

In [ ]:
np.save('/content/drive/My Drive/prediction_monogram.npy',prediction)
